In [563]:
from src import DATA_DIR
import pandas as pd
from itertools import groupby
from collections import Counter
from hazm import Normalizer, sent_tokenize, word_tokenize
from hashlib import sha256
import copy
from collections import defaultdict

##### 
### Read Files

In [564]:
data_file_name_list = ['روغن موتور.csv',
                       'تعویض روغنی.csv',
                       'روغن ماشین.csv',
                      ]

In [565]:
df = pd.read_csv(DATA_DIR / 'data' / data_file_name_list[0])

for path in data_file_name_list[1:]:
    df = pd.concat([df, pd.read_csv(DATA_DIR / 'data' / path)])

df = df.drop_duplicates(subset = "Tweet_ID")

_ = pd.read_csv(DATA_DIR / 'Input/Themes.csv')
themes = {_.iloc[i, 1]:[sha256(_.iloc[i, 0].encode()).hexdigest()[:18], _.iloc[i, 0].replace(" ", "_")] for i in range(len(_))}

In [566]:
# for i in list(df[df.Text.str.contains('عوض') * df.Text.str.contains('تومن')].Text):
#     print(i)
#     print("-"*50)

In [567]:
# Counter(" ".join(df[df.Text.str.contains('مکانیک')].Text).split()).most_common()

In [568]:
delete_id = ['nilaka',
             'مارلیک',
             'baryar',
             'arazoilco',
            ]

delete_tweet = ['گوجه',
                'مایع',
                'خوراکی',
                'گوشت',
                'مرغ',
                'جمهوری',
                'غذا',
                ['گرون', 'تقلبی'],
                ['گران', 'تقلبی'],
                ['گرون', 'فیک'],
                ['گران', 'فیک'],
               ]

for id_ in delete_id:
    df = df[~df.User_ID.str.contains(id_)]
    
for item in delete_tweet:
    if isinstance(item, str):
        df = df[~df.Text.str.contains(item)]

    elif isinstance(item, list):
        rem = df.Text.str.contains(item[0])
        for j in item[1:]:
            rem = rem & df.Text.str.contains(j)
            
        df = df[~rem]
    
tweet_texts = ' Oadf6dsaf8dfa '.join(list(df.Text))

tweet_texts2 = copy.deepcopy(tweet_texts)

###### 
### Detect Themes

In [569]:
# Thenes

for word_, theme_ in themes.items():
    tweet_texts = tweet_texts.replace(word_, theme_[0])

for theme_ in themes.values():
    tweet_texts = tweet_texts.replace(theme_[0], theme_[1])

filt = [*[item[1] for item in list(themes.values())], 'Oadf6dsaf8dfa']

_ = list(filter(lambda x: x in filt, tweet_texts.split()))
tweet_list = [list(group) for k, group in groupby(_, lambda x: x == "Oadf6dsaf8dfa") if not k]

_ = [(item[0], item[0].replace("_", " "), item[1]) for item in Counter(_).most_common()[1:]]
df_output_nodes = pd.DataFrame(_, columns=['Id', 'Label', 'Weight'])

In [570]:
edges = defaultdict(int)

for tweet in tweet_list:
    set_ = set()
    if len(tweet) <=1:
        continue
    
    for i in range(len(tweet)):
        for j in range(i+1, len(tweet)):
            if tweet[i] != tweet[j]:
                if (tweet[i], tweet[j]) in set_:
                    continue
                
                else:
                    edges[(tweet[i], tweet[j])] += 1
                    set_.add((tweet[i], tweet[j]))

In [571]:
# Words

filt = [*list(themes.keys()), 'Oadf6dsaf8dfa']
_ = list(filter(lambda x: x in filt, tweet_texts2.split()))
tweet_list_word = [list(group) for k, group in groupby(_, lambda x: x == "Oadf6dsaf8dfa") if not k]

In [572]:
edges_word = defaultdict(int)

for tweet in tweet_list_word:
    set_ = set()
    if len(tweet) <=1:
        continue
    
    for i in range(len(tweet)):
        for j in range(i+1, len(tweet)):
            if tweet[i] != tweet[j]:
                if (tweet[i], tweet[j]) in set_:
                    continue
                
                else:
                    edges_word[(tweet[i], tweet[j])] += 1
                    set_.add((tweet[i], tweet[j]))

###### 
### Save Files

In [573]:
tweet_list = []
tweet_list_word = []

for key, value in edges.items():
    tweet_list.append((*key, value))
    
for key, value in edges_word.items():
    tweet_list_word.append((*key, value))

df_output_edges = pd.DataFrame(tweet_list, columns=['Source', 'Target', 'Weight'])
df_output_edges.to_csv(DATA_DIR / 'data/Gephi/edges.csv', index=False, encoding="utf-8-sig")
df_output_nodes.to_csv(DATA_DIR / 'data/Gephi/nodes.csv', index=False, encoding="utf-8-sig")

df_output_edges_word = pd.DataFrame(tweet_list_word, columns=['Source', 'Target', 'Weight'])
df_output_edges_word = df_output_edges_word.sort_values('Weight', ascending=False)
df_output_edges_word.to_csv(DATA_DIR / 'data/Gephi/edges_word.csv', index=False, encoding="utf-8-sig")

###### 
### Most Common Words

In [574]:
def remove_stopwords(text: str, stopwords: list):
    """
    :param text: text you want to delete stopwords from dat
    :param stopwords: list of stopwords
    """
    tokens = word_tokenize(text)
    tokens = filter(lambda word: word not in stopwords, tokens)
    return " ".join(tokens)

In [575]:
with open(DATA_DIR / 'Input/persian_stop_words.txt') as fp:
    stopwords = fp.read()

stopwords = stopwords.split()

In [576]:
list_ = list(df.Text)
list_ = [str(text) for text in list_]
text = ' '.join(list_)
text = remove_stopwords(text, stopwords)
text = text.split()

In [577]:
from collections import Counter
df = pd.DataFrame(Counter(text).most_common(), columns=['word', 'count'])
df.to_csv(DATA_DIR / 'Input/most_common_words.csv', index=False, encoding="utf-8-sig")